In [ ]:
#Prep Data (No1)
import os
import sys
import json

master_file = "/content/drive/MyDrive/sepsismodel/file/master.csv"

def gen_master_feature_list():
    # master information
    m_set = set()
    for i_line,line in enumerate(open(master_file)):
        if i_line != 0:
            data = line.strip().split(',')
            for i,d in enumerate(data[1:]):
                m_set.add(str(i) + d)
    return sorted(m_set)

def gen_patient_master_dict(master_list):
    patient_master_dict = dict()
    # master information
    master_set = [set() for _ in range(6)]
    for i_line,line in enumerate(open(master_file)):
        if i_line != 0:
            data = line.strip().split(',')
            patient = data[0]
            feature = ['0' for _ in range(43)]
            for i,d in enumerate(data[1:]):
                m = str(i) + d
                idx = master_list.index(m)
                feature[idx] = '1'
            patient_master_dict[patient] = ''.join(feature)
    content = json.dumps(patient_master_dict,indent=4,ensure_ascii=False)
    with open("/content/drive/MyDrive/sepsismodel/runfiles/patient_master_dict.json",'w') as f:
        f.write(content)

def main():
    master_list = gen_master_feature_list()
    gen_patient_master_dict(master_list)

if __name__ == '__main__':
    main()

In [ ]:
master_file = "/content/drive/MyDrive/sepsismodel/file/master.csv"

for i_line,line in enumerate(open(master_file)):
  if i_line in (0,1):
    print(line)
  else:
    break

adm_id,gender,race,admission_type,admission_source,care_setting,age_grp

A100001,Male,African American,Emergency,Emergency Room,Care Setting Undefined,70~80



In [ ]:
#Prep Data (No2)
def gen_patient_time_dict():
    vital_file = "/content/drive/MyDrive/sepsismodel/file/vital.csv"
    patient_time_dict = dict()
    for i_line,line in enumerate(open(vital_file)):
        if 'event_time' not in line:
            patient, time = line.strip().split(',')[:2]
            patient_time_dict[patient] = max(patient_time_dict.get(patient, 0), float(time))
    content = json.dumps(patient_time_dict,indent=4,ensure_ascii=False)
    with open("/content/drive/MyDrive/sepsismodel/runfiles/patient_time_dict.json",'w') as f:
        f.write(content)

def main():
    gen_patient_time_dict()

if __name__ == '__main__':
    main()

In [ ]:
#Prep Data (No3)
def gen_feature_order_dict():
    '''
    generate the order of value for each feature
    '''

    feature_value_order_dict = dict()

    # vital information
    vital_file = "/content/drive/MyDrive/sepsismodel/file/vital.csv"
    vital_dict = { } # key-valuelist-dict
    for i_line,line in enumerate(open(vital_file)):
        if i_line % 10000 == 0:
            print(i_line)
        # if i_line > 10000:
        #     break
        if i_line == 0:
            new_line = ''
            vis = 0
            for c in line:
                if c == '"':
                    vis = (vis + 1) % 2
                if vis == 1 and c == ',':
                    c = ';'
                new_line += c
            line = new_line
            col_list = line.strip().split(',')[1:]
            for col in col_list:
                vital_dict[col] = []
        else:
            ctt_list = line.strip().split(',')[1:]
            assert len(ctt_list) == len(col_list)
            for col,ctt in zip(col_list, ctt_list):
                if len(ctt):
                    vital_dict[col].append(float(ctt))
        # if i_line > 10000:
        #    break
        # if i_line % 10000 == 0:
        #     print i_line

    # add group info
    with open("/content/drive/MyDrive/sepsismodel/file/similar.json",'r') as f:
        groups = json.loads(f.read())
        f.close()
    with open("/content/drive/MyDrive/sepsismodel/file/feature_index_dict.json",'r') as g:
        feature_index_dict = json.loads(g.read())
        g.close()
    with open("/content/drive/MyDrive/sepsismodel/file/index_feature_list.json",'r') as h:
        index_feature_list = json.loads(h.read())
        h.close()
    for g in groups:
        for k in g:
            mg = min(g)
            if k != mg:
                kf = index_feature_list[k]
                mf = index_feature_list[mg]
                print(kf,mf)
                try:
                  vital_dict[mf] = vital_dict[mf] + vital_dict[kf]
                  vital_dict.pop(kf)
                except:
                  pass
    print('features', len(vital_dict))

    # feature_count_dict = { k: len(v) for k,v in vital_dict.items() }
    # py_op.mywritejson(os.path.join(args.file_dir, 'feature_count_dict.json'), feature_count_dict)

    ms_list = []
    for col in col_list:
        if col not in vital_dict:
            continue
        value_list = sorted(vital_dict[col])
        value_order_dict = dict()
        value_minorder_dict = dict()
        value_maxorder_dict = dict()
        for i_value, value in enumerate(value_list):
            if value not in value_minorder_dict:
                value_minorder_dict[value] = i_value
            if value == value_list[-1]:
                value_maxorder_dict[value] = len(value_list) - 1
                break
            if value != value_list[i_value+1]:
                value_maxorder_dict[value] = i_value
        for value in value_maxorder_dict:
            value_order_dict[value] = (value_maxorder_dict[value] + value_minorder_dict[value]) / 2.0 / len(value_list)
        feature_value_order_dict[col] = value_order_dict
    content = json.dumps(feature_value_order_dict,indent=4,ensure_ascii=False)
    with open("/content/drive/MyDrive/sepsismodel/runfiles/feature_value_order_dict.json",'w') as f:
        f.write(content)

def main():
    gen_feature_order_dict()

if __name__ == '__main__':
    #os.system('clear')
    main()


0
Base Excess Calc Base Excess
"Base Excess Calc; Arterial" Base Excess
"CO2 Total; Arterial" CO2
"CO2 Total; Serum" CO2
"Calcium; Serum" Calcium Quant
"Glomerular Filtration Rate; Est" Glomerular Filtration Rate
Glucose WBlood Quant Glucose Stick/Meter WBlood POC
"HCO3; Arterial" HCO3
HR Apical HR
HR Monitored HR
"Lymphocytes Cnt Bld Auto;Abs" Lymph Abs Cnt
"Lymphocytes Cnt Bld Auto;%" Lymph %
Lymphocytes NFr Bld Auto Lymph %
"Magnesium; Serum/Plasma" Magnesium
"PCO2; Arterial" PCO2
"PO2; Arterial" PO2
PaO2 PO2
PT Time PPP PT
Pulse Peripheral Pulse
Resp Rt Tot Resp Rt
SaO2 SO2
"SaO2 %; Arterial" SO2
Temp Axillary Temp
Temp Oral Temp
Temp Temporal Artery Temp
Temp Tympanic Temp
"Albumin; Serum" Albumin Quant
Neutrophil Seg Neutrophil %
"Nucleated RBC Ratio; Blood Auto" Diff Nucleated RBC
features 116


In [ ]:
#Prep Data (No4)
def gen_json_data():
    vital_file = "/content/drive/MyDrive/sepsismodel/file/vital.csv"
    patient_time_record_dict = dict()
    with open("/content/drive/MyDrive/sepsismodel/file/feature_index_dict.json",'r') as h:
        feature_index_dict = json.loads(h.read())
        h.close()
    with open("/content/drive/MyDrive/sepsismodel/file/feature_value_order_dict.json",'r') as f:
        feature_value_order_dict = json.loads(f.read())
        f.close()
    feature_value_order_dict = { str(feature_index_dict[k]):v for k,v in feature_value_order_dict.items()  if 'event' not in k}
    with open("/content/drive/MyDrive/sepsismodel/file/index_group_dict.json",'r') as g:
        index_group_dict = json.loads(g.read())
        g.close()
    with open("/content/drive/MyDrive/sepsismodel/runfiles/patient_time_dict.json",'r') as i:
        patient_time_dict = json.loads(i.read())
        i.close()
    mx_time = - 100
    for i_line, line in enumerate(open(vital_file)):
        if i_line % 10000 == 0:
            print('line', i_line)
        if 'event_time' not in line:
            data = line.strip().split(',')
            patient, time = data[:2]
            time = int(float(time))
            mx_time = max(mx_time, time)
            if patient not in patient_time_record_dict:
                patient_time_record_dict[patient] = dict()
            if time not in patient_time_record_dict[patient]:
                patient_time_record_dict[patient][time] = dict()

            data = data[2:]
            vs = dict()
            for idx, val in enumerate(data):
                if len(val) == 0:
                    continue
                if str(idx) in index_group_dict:
                    idx = index_group_dict[str(idx)]
                value_order = feature_value_order_dict[str(idx)]
                try:
                  vs[idx] = value_order[val]
                except:
                  pass
            patient_time_record_dict[patient][time].update(vs)

    new_d = dict()
    for p, tr in patient_time_record_dict.items():
        new_d[p] = dict()
        for t, vs in tr.items():
            if mx_time > 0:
                t = int(t - patient_time_dict[p] - 4)
            if t < - 102:
                continue
            nvs = []
            for k in sorted(vs.keys()):
                nvs.append([k, vs[k]])
            new_d[p][t] = nvs
    with open("/content/drive/MyDrive/sepsismodel/runfiles/patient_time_record_dict.json", 'w') as f:
        f.write(json.dumps(new_d))

def main():
    gen_json_data()

if __name__ == '__main__':
    main()


line 0


In [ ]:
#Prep Data (No5)
def gen_patient_label_dict():
    patient_label_dict = dict()
    label_file = "/content/drive/MyDrive/sepsismodel/file/label.csv"
    for i_line,line in enumerate(open(label_file)):
        if i_line != 0:
            data = line.strip().split(',')
            patient = data[0]
            label  = data[-1]
            patient_label_dict[patient] = int(label)
    with open("/content/drive/MyDrive/sepsismodel/runfiles/patient_label_dict.json", 'w') as f:
        f.write(json.dumps(patient_label_dict))
        f.close()

def main():
    gen_patient_label_dict()

if __name__ == '__main__':
    main()


In [1]:
#Set up arguments

import argparse
parser = argparse.ArgumentParser(description='SepsisModel')
parser.add_argument('-dd',
        '--data-dir',
        type=str,
        default='/content/drive/MyDrive/sepsismodel/',
        help='data directory'
        )
parser.add_argument(
        '--result-dir',
        type=str,
        default='../result/',
        help='result directory'
        )
parser.add_argument(
        '--file-dir',
        type=str,
        default='../file/',
        help='useful file directory'
        )
parser.add_argument(
        '--vital-file',
        type=str,
        default='../file/vital.csv',
        help='vital information'
        )
parser.add_argument(
        '--master-file',
        type=str,
        default='../file/master.csv',
        help='master information'
        )
parser.add_argument(
        '--label-file',
        type=str,
        default='../file/label.csv',
        help='label'
        )
parser.add_argument(
        '--model',
        '-m',
        type=str,
        default='lstm',
        help='model'
        )
parser.add_argument(
        '--embed-size',
        metavar='EMBED SIZE',
        type=int,
        default=256,
        help='embed size'
        )
parser.add_argument(
        '--rnn-size',
        metavar='rnn SIZE',
        type=int,
        help='rnn size'
        )
parser.add_argument(
        '--hidden-size',
        metavar='hidden SIZE',
        type=int,
        help='hidden size'
        )
parser.add_argument(
        '--split-num',
        metavar='split num',
        type=int,
        default=5,
        help='split num'
        )
parser.add_argument(
        '--split-nor',
        metavar='split normal range',
        type=int,
        default=3,
        help='split num'
        )
parser.add_argument(
        '--num-layers',
        metavar='num layers',
        type=int,
        default=2,
        help='num layers'
        )
parser.add_argument(
        '--num-code',
        metavar='num codes',
        type=int,
        default=1200,
        help='num code'
        )
parser.add_argument(
        '--use-glp',
        metavar='use global pooling operation',
        type=int,
        default=1,
        help='use global pooling operation'
        )
parser.add_argument(
        '--use-visit',
        metavar='use visit as input',
        type=int,
        default=1,
        help='use visit as input'
        )
parser.add_argument(
        '--use-value',
        metavar='use value embedding as input',
        type=int,
        default=1,
        help='use value embedding as input'
        )
parser.add_argument(
        '--use-cat',
        metavar='use cat for time and value embedding',
        type=int,
        default=1,
        help='use cat or add'
        )
parser.add_argument(
        '--use-trend',
        metavar='use feature variation trend',
        type=int,
        default=1,
        help='use trend'
        )
parser.add_argument(
        '--avg-time',
        metavar='avg time for trend, hours',
        type=int,
        default=4,
        help='avg time for trend'
        )
parser.add_argument(
        '--seed',
        metavar='seed',
        type=int,
        default=1,
        help='seed'
        )
parser.add_argument(
        '--set',
        metavar='split set for training',
        type=int,
        default=0,
        help='split set'
        )
parser.add_argument(
        '--last-time',
        metavar='last-time for task2',
        type=int,
        default=-4,
        help='last time'
        )
parser.add_argument(
        '--final',
        metavar='final test to submit',
        type=int,
        default=0,
        help='final'
        )

parser.add_argument('--phase',
        default='train',
        type=str,
        metavar='S',
        help='pretrain/train/test phase')
parser.add_argument(
        '--batch-size',
        '-b',
        metavar='BATCH SIZE',
        type=int,
        default=32,
        help='batch size'
        )
parser.add_argument('--save-dir',
        default='../../data',
        type=str,
        metavar='S',
        help='save dir')
parser.add_argument('--resume',
        default='',
        type=str,
        metavar='S',
        help='start from checkpoints')
parser.add_argument('--task',
        default='task2',
        type=str,
        metavar='S',
        help='start from checkpoints')

#####
parser.add_argument('-j',
        '--workers',
        default=8,
        type=int,
        metavar='N',
        help='number of data loading workers (default: 32)')
parser.add_argument('--lr',
        '--learning-rate',
        default=0.0001,
        type=float,
        metavar='LR',
        help='initial learning rate')
parser.add_argument('--epochs',
        default=20,
        type=int,
        metavar='N',
        help='number of total epochs to run')
parser.add_argument('--save-freq',
        default='5',
        type=int,
        metavar='S',
        help='save frequency')
parser.add_argument('--save-pred-freq',
        default='10',
        type=int,
        metavar='S',
        help='save pred clean frequency')
parser.add_argument('--val-freq',
        default='5',
        type=int,
        metavar='S',
        help='val frequency')
args, unknown = parser.parse_known_args()

args.hard_mining = 0
args.gpu = 1
args.use_trend = max(args.use_trend, args.use_value)
args.use_value = max(args.use_trend, args.use_value)
args.rnn_size = args.embed_size
args.hidden_size = args.embed_size

In [2]:
#Function.py

def save_model(p_dict, name='best.ckpt', folder='/content/drive/MyDrive/sepsismodel/result/models/'):
    args = p_dict['args']
    name = '{:s}-snm-{:d}-snr-{:d}-value-{:d}-trend-{:d}-cat-{:d}-lt-{:d}-size-{:d}-seed-{:d}-{:s}'.format(args.task, 
            args.split_num, args.split_nor, args.use_value, args.use_trend, 
            args.use_cat, args.last_time, args.embed_size, args.seed, name)
    if not os.path.exists(folder):
        os.mkdir(folder)
    model = p_dict['model']
    state_dict = model.state_dict()
    for key in state_dict.keys():
        state_dict[key] = state_dict[key].cpu()
    all_dict = {
            'epoch': p_dict['epoch'],
            'args': p_dict['args'],
            'best_metric': p_dict['best_metric'],
            'state_dict': state_dict 
            }
    torch.save(all_dict, os.path.join(folder, name))

def load_model(p_dict, model_file):
    all_dict = torch.load(model_file)
    p_dict['epoch'] = all_dict['epoch']
    # p_dict['args'] = all_dict['args']
    p_dict['best_metric'] = all_dict['best_metric']
    # for k,v in all_dict['state_dict'].items():
    #     p_dict['model_dict'][k].load_state_dict(all_dict['state_dict'][k])
    p_dict['model'].load_state_dict(all_dict['state_dict'])


def save_segmentation_results(images, segmentations, folder='../data/middle_segmentation'):
    stride = args.stride

    if not os.path.exists(folder):
        os.mkdir(folder)

    # images = images.data.cpu().numpy()
    # segmentations = segmentations.data.cpu().numpy()
    images = (images * 128) + 127
    segmentations[segmentations>0] = 255
    segmentations[segmentations<0] = 0

    # print(images.shape, segmentations.shape)
    for ii, image, seg in zip(range(len(images)), images, segmentations):
        image = data_function.numpy_to_image(image)
        new_seg = np.zeros([3, seg.shape[1] * stride, seg.shape[2] * stride])
        for i in range(seg.shape[1]):
            for j in range(seg.shape[2]):
                for k in range(3):
                    new_seg[k, i*stride:(i+1)*stride, j*stride:(j+1)*stride] = seg[0,i,j]
        seg = new_seg
        seg = data_function.numpy_to_image(seg)
        image.save(os.path.join(folder, str(ii) + '_image.png'))
        seg.save(os.path.join(folder, str(ii) + '_seg.png'))


def save_middle_results(data, folder = '../data/middle_images'):
    stride = args.stride

    if not os.path.exists(folder):
        os.mkdir(folder)
    numpy_data = [x.data.numpy() for x in data[1:]]
    data =  data[:1] + numpy_data
    image_names, images, word_labels, seg_labels, bbox_labels, bbox_images =  data[:6]
    images = (images * 128) + 127
    seg_labels = seg_labels*127 + 127


    for ii, name, image, seg, bbox_image in zip(range(len(image_names)), image_names, images, seg_labels, bbox_images):
        name = name.split('/')[-1]
        image = data_function.numpy_to_image(image)
        new_seg = np.zeros([3, seg.shape[1] * stride, seg.shape[2] * stride])
        # print(seg[0].max(),seg[0].min())
        for i in range(seg.shape[1]):
            for j in range(seg.shape[2]):
                for k in range(3):
                    new_seg[k, i*stride:(i+1)*stride, j*stride:(j+1)*stride] = seg[0,i,j]
        seg = new_seg
        seg = data_function.numpy_to_image(seg)
        # image.save(os.path.join(folder, name))
        # seg.save(os.path.join(folder, name.replace('image.png', 'seg.png')))
        image.save(os.path.join(folder, str(ii) + '_image.png'))
        seg.save(os.path.join(folder, str(ii) + '_seg.png'))

        for ib,bimg in enumerate(bbox_image):
            # print(bimg.max(), bimg.min(), bimg.dtype)
            bimg = data_function.numpy_to_image(bimg)
            bimg.save(os.path.join(folder, str(ii)+'_'+ str(ib) + '_bbox.png'))

def save_detection_results(names, images, detect_character_output, folder='../data/test_results/'):
    stride = args.stride

    if not os.path.exists(folder):
        os.mkdir(folder)
    # images = images.data.cpu().numpy()                                      # [bs, 3, w, h]
    images = (images * 128) + 127
    # detect_character_output = detect_character_output.data.cpu().numpy()    # [bs, w, h, n_anchors, 5+class]

    for i, name, image, bboxes in zip(range(len(names)), names, images, detect_character_output):
        name = name.split('/')[-1]

        ### 保存原图
        # data_function.numpy_to_image(image).save(os.path.join(folder, name))
        data_function.numpy_to_image(image).save(os.path.join(folder, str(i) + '_image.png'))

        detected_bbox = detect_function.nms(bboxes)
        # print([b[-1] for b in detected_bbox])
        # print(len(detected_bbox))
        image = data_function.add_bbox_to_image(image, detected_bbox)
        # image.save(os.path.join(folder, name.replace('.png', '_bbox.png')))
        image.save(os.path.join(folder, str(i) + '_bbox.png'))



def compute_detection_metric(outputs, labels, loss_outputs,metric_dict):
    loss_outputs[0] = loss_outputs[0].data
    metric_dict['metric'] = metric_dict.get('metric', []) + [loss_outputs]

def compute_segmentation_metric(outputs, labels, loss_outputs, metric_dict):
    loss_outputs[0] = loss_outputs[0].data
    metric_dict['metric'] = metric_dict.get('metric', []) + [loss_outputs]

def compute_metric(outputs, labels, time, loss_outputs,metric_dict, phase='train'):
    # loss_output_list, f1score_list, recall_list, precision_list):
    if phase != 'test':
        preds = outputs.data.cpu().numpy()
        labels = labels.data.cpu().numpy()
    else:
        preds = np.array(outputs)

    preds = preds.reshape(-1)
    labels = labels.reshape(-1)

    if time is not None:
        time = time.reshape(-1)
        assert preds.shape == time.shape
        time = time[labels>-0.5]
    assert preds.shape == labels.shape

    preds = preds[labels>-0.5]
    label = labels[labels>-0.5]

    pred = preds > 0

    assert len(pred) == len(label)

    tp = (pred + label == 2).sum()
    tn = (pred + label == 0).sum()
    fp = (pred - label == 1).sum()
    fn = (pred - label ==-1).sum()
    fp = (pred - label == 1).sum()

    metric_dict['tp'] = metric_dict.get('tp', 0.0) + tp
    metric_dict['tn'] = metric_dict.get('tn', 0.0) + tn
    metric_dict['fp'] = metric_dict.get('fp', 0.0) + fp
    metric_dict['fn'] = metric_dict.get('fn', 0.0) + fn
    loss = []
    for x in loss_outputs:
        if x == 0:
            loss.append(x)
        else:
            loss.append(x.data.cpu().numpy())
    # loss = [[x.data.cpu().numpy() for x in loss_outputs]]
    metric_dict['loss'] = metric_dict.get('loss', []) +  [loss]
    if phase != 'train':
        metric_dict['preds'] = metric_dict.get('preds', []) + list(preds)
        metric_dict['labels'] = metric_dict.get('labels', []) + list(label)
        if time is not None:
            metric_dict['times'] = metric_dict.get('times', []) + list(time)

def compute_metric_multi_classification(outputs, labels, loss_outputs, metric_dict):
    preds = outputs.data.cpu().numpy() > 0
    labels = labels.data.cpu().numpy()
    for pred, label in zip(preds, labels):
        pred = np.argmax(pred)
        tp = (pred == label ).sum()
        fn = (pred != label).sum()
        accuracy = 1.0 * tp / (tp + fn)
        metric_dict['accuracy'] = metric_dict.get('accuracy', []) + [accuracy]
    metric_dict['loss'] = metric_dict.get('loss', []) +  [[x.data.cpu().numpy() for x in loss_outputs]]


def print_metric(first_line, metric_dict, phase='train'):
    print(first_line)
    loss_array = np.array(metric_dict['loss']).mean(0)
    tp = metric_dict['tp']
    tn = metric_dict['tn']
    fp = metric_dict['fp']
    fn = metric_dict['fn']
    accuracy = 1.0 * (tp + tn) / (tp + tn + fp + fn)
    recall = 1.0 * tp / (tp + fn + 10e-20)
    precision = 1.0 * tp / (tp + fp + 10e-20)
    f1score = 2.0 * recall * precision / (recall + precision + 10e-20)


    
    loss_array = loss_array.reshape(-1)

    print('loss: {:3.4f}\t pos loss: {:3.4f}\t negloss: {:3.4f}'.format(loss_array[0], loss_array[1], loss_array[2]))
    print('accuracy: {:3.4f}\t f1score: {:3.4f}\t recall: {:3.4f}\t precision: {:3.4f}'.format(accuracy, f1score, recall, precision))
    print('\n')

    if phase != 'train':
        fpr, tpr, thr = metrics.roc_curve(metric_dict['labels'], metric_dict['preds'])
        return metrics.auc(fpr, tpr)
    else:
        return f1score

# def load_all():
#     fo = '/content/drive/MyDrive/sepsismodel/code/models'
#     pre = ''
#     for fi in sorted(os.listdir(fo)):
#         if fi[:5] != pre:
#             print
#             pre = fi[:5]
#         print(os.path.join(fo, fi))
#         x = torch.load(os.path.join(fo, fi))
#         print (x['best_metric'], fi)
# load_all()



In [3]:


import os
import json
import torch
from torch import nn
import torch.nn.functional as F
from torch.autograd import *

import numpy as np

import sys
os.chdir('/content/drive/MyDrive/sepsismodel/code/tools/')
import parse, py_op
#args = parse.args

args.hard_mining = 0
args.gpu = 1
args.use_trend = max(args.use_trend, args.use_value)
args.use_value = max(args.use_trend, args.use_value)
args.rnn_size = args.embed_size
args.hidden_size = args.embed_size

def time_encoding_data(d = 512, time = 200):
    vec = np.array([np.arange(time) * i for i in range(int(d/2))], dtype=np.float32).transpose()
    vec = vec / vec.max() / 2
    encoding = np.concatenate((np.sin(vec), np.cos(vec)), 1)
    encoding = torch.from_numpy(encoding)
    return encoding


class LSTM(nn.Module):
    def __init__(self, opt):
        super ( LSTM, self ).__init__ ( )
        self.use_cat = args.use_cat
        self.avg_time = args.avg_time

        self.embedding = nn.Embedding (opt.vocab_size, opt.embed_size )
        self.lstm = nn.LSTM ( input_size=opt.embed_size,
                              hidden_size=opt.hidden_size,
                              num_layers=opt.num_layers,
                              batch_first=True,
                              bidirectional=True)

        self.linear_embed = nn.Sequential (
            nn.Linear ( opt.embed_size, opt.embed_size ),
            nn.ReLU ( ),
            nn.Linear ( opt.embed_size, opt.embed_size ),
        )
        self.tv_mapping = nn.Sequential (
            nn.Linear ( opt.embed_size , int(opt.embed_size / 2)),
            nn.ReLU ( ),
            nn.Dropout ( 0.25 ),
            nn.Linear ( int(opt.embed_size / 2), opt.embed_size ),
        )
        self.alpha = nn.Linear(args.embed_size, 1)


        no = 1
        if self.use_cat:
            no += 1
        self.output_time = nn.Sequential (
                nn.Linear(opt.embed_size * no, opt.embed_size),
                nn.ReLU ( ),
        )

        time = 200
        self.time_encoding = nn.Embedding.from_pretrained(time_encoding_data(opt.embed_size, time))
        self.time_mapping = nn.Sequential (
            nn.Linear ( opt.embed_size, opt.embed_size),
            nn.ReLU ( ),
            nn.Dropout ( 0.25 ),
            nn.Linear ( opt.embed_size, opt.embed_size)
            )

        self.embed_linear = nn.Sequential (
            nn.Linear ( opt.embed_size, opt.embed_size),
            nn.ReLU ( ),
            # nn.Dropout ( 0.25 ),
            # nn.Linear ( opt.embed_size, opt.embed_size),
            # nn.ReLU ( ),
            nn.Dropout ( 0.25 ),
        )
        self.relu = nn.ReLU ( )

        self.linears = nn.Sequential (
            nn.Linear ( opt.hidden_size * 2, opt.rnn_size ),
            # nn.ReLU ( ),
            # nn.Dropout ( 0.25 ),
            # nn.Linear ( opt.rnn_size, opt.rnn_size ),
            nn.ReLU ( ),
            nn.Dropout ( 0.25 ),
            nn.Linear ( opt.rnn_size, 1),
        )
        mn = 128
        self.master_linear = nn.Sequential (
            nn.Linear ( 43, mn),
            # nn.ReLU ( ),
            # nn.Dropout ( 0.25 ),
            # nn.Linear ( mn, mn),
            nn.ReLU ( ),
            nn.Dropout ( 0.25 ),
            nn.Linear ( mn, 1),
        )
        self.output = nn.Sequential (
            nn.Linear ( mn + opt.rnn_size , opt.rnn_size),
            nn.ReLU ( ),
            nn.Linear ( opt.rnn_size, mn),
            nn.ReLU ( ),
            nn.Dropout ( 0.25 ),
            nn.Linear ( mn, 1),
        )
        self.pooling = nn.AdaptiveMaxPool1d(1)
        self.opt = opt

    def visit_pooling(self, x, mask, time, value=None, trend=None):

        output = x
        size = output.size()
        output = output.view(size[0] * size[1], size[2], output.size(3)) # (bs*98, 72, 512)
        if args.use_glp:
            output = torch.transpose(output, 1,2).contiguous() # (bs*98, 512, 72)
            output = self.pooling(output)
        else:
            weight = self.alpha(output) # (bs*98, 72, 1)
            # print weight.size()
            weight = weight.view(size[0]*size[1], size[2])
            # print weight.size()
            weight = F.softmax(weight)
            x = weight.data.cpu().numpy()
            # print x.shape
            weight = weight.view(size[0]*size[1], size[2], 1).expand(output.size())
            output = weight * output # (bs*98, 512, 72)
            # print output.size()
            output = output.sum(1)
            # print output.size()
            # output = torch.transpose(output, 1,2).contiguous() 
        output = output.view(size[0], size[1], size[3])

        # time encoding
        time = - time.long()
        time = self.time_encoding(time)
        time = self.time_mapping(time)

        if self.use_cat:
            output = torch.cat((output, time), 2) 
            output = self.relu(output)
            output = self.output_time(output) 
        else:
            output = output + time
            output = self.relu(output)



        return output


    def forward_2(self, x, master, mask=None, time=None, phase='train', value=None, trend=None):
        '''
        task2
        '''
        size = list(x.size())
        x = x.view(-1)
        x = self.embedding( x )
        x = self.embed_linear(x)
        size.append(-1)
        x = x.view(size)
        if mask is not None:
            x = self.visit_pooling(x, mask, time, value, trend)
        lstm_out, _ = self.lstm( x )
        lstm_out = torch.transpose(lstm_out, 1, 2).contiguous() # (bs, 512, 98)
        mask = self.pooling(mask)
        # print 'mask', mask.size()
        pool_out = []
        mask_out = []
        time_out = []
        time = time.data.cpu().numpy()
        if phase == 'train':
            start, delta = 4, 6
        else:
            start, delta = 1, 1
        for i in range(start, lstm_out.size(2), delta):
            pool_out.append(self.pooling(lstm_out[:,:, :i]))
            mask_out.append(mask[:, i])
            time_out.append(time[:, i])
        pool_out.append(self.pooling(lstm_out))
        mask_out.append(mask[:, 0])
        time_out.append(np.zeros(size[0]) - 4)

        lstm_out = torch.cat(pool_out, 2)  # (bs, 512, 98)
        mask_out = torch.cat(mask_out, 1)  # (bs, 98)
        time_out = np.array(time_out).transpose() # (bs, 98)

        # print 'lstm_out', lstm_out.size()
        # print 'mask_out', mask_out.size()
        # print err

        lstm_out = torch.transpose(lstm_out, 1, 2).contiguous() # (bs, 98, 512)

        out_vital = self.linears(lstm_out)
        size = list(out_vital.size())
        out_vital = out_vital.view(size[:2])
        out_master = self.master_linear(master).expand(size[:2])
        out = out_vital + out_master
        return out, mask_out, time_out

    def forward_1(self, x, master, mask=None, time=None, phase='train', value=None, trend=None):
        # out = self.master_linear(master)
        size = list(x.size())
        x = x.view(-1)
        x = self.embedding( x )
        # print x.size()
        x = self.embed_linear(x)
        size.append(-1)
        x = x.view(size)
        if mask is not None:
            x = self.visit_pooling(x, mask, time, value, trend)
        lstm_out, _ = self.lstm( x )

        lstm_out = torch.transpose(lstm_out, 1, 2).contiguous()
        lstm_out = self.pooling(lstm_out)
        lstm_out = lstm_out.view(lstm_out.size(0), -1)

        out = self.linears(lstm_out) + self.master_linear(master)
        return out

    def forward(self, x, master, mask=None, time=None, phase='train', value=None, trend=None):
        if args.task == 'task2':
            return self.forward_2(x, master, mask, time, phase, value, trend)
            # return self.forward_1(x, master, mask, time, phase, value, trend)
        else:
            return self.forward_1(x, master, mask, time, phase, value, trend)

In [4]:
#Main run (No6)
import os
import sys
import time
import json
import traceback
import numpy as np
from glob import glob
from tqdm import tqdm
from sklearn import metrics


# torch
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
os.chdir('/content/drive/MyDrive/sepsismodel/code/tools/')
#import parse, 
import py_op

os.chdir('/content/drive/MyDrive/sepsismodel/code')
import loss
#import function

# import framework
os.chdir('/content/drive/MyDrive/sepsismodel/code/loaddata')
import dataloader

os.chdir('/content/drive/MyDrive/sepsismodel/code/models')
import lstm

def train_eval(p_dict, phase='train'):
    epoch = p_dict['epoch']
    model = p_dict['model']           
    loss = p_dict['loss']             
    if phase == 'train':
        data_loader = p_dict['train_loader']        
        optimizer = p_dict['optimizer']             
    else:
        data_loader = p_dict['val_loader']

    classification_metric_dict = dict()
    # if args.task == 'case1':

    for i,data in enumerate(tqdm(data_loader)):
        if args.use_visit:
            if args.gpu:
                data = [ Variable(x.cuda()) for x in data ]
            visits, values, mask, master, labels, times, trends  = data
            if i == 0:
                print('input size', visits.size())
            output = model(visits, master, mask, times, phase, values, trends)
        else:
            inputs = Variable(data[0].cuda())
            labels = Variable(data[1].cuda())
            output = model(inputs)

        # if 0:
        if args.task == 'task2':
            output, mask, time = output
            labels = labels.unsqueeze(-1).expand(output.size()).contiguous()
            labels[mask==0] = -1
        else:
            time = None

        classification_loss_output = loss(output, labels, args.hard_mining)
        loss_gradient = classification_loss_output[0]
        compute_metric(output, labels, time, classification_loss_output, classification_metric_dict, phase)

        # print(outputs.size(), labels.size(),data[3].size(),segment_line_output.size())
        # print('detection', detect_character_labels.size(), detect_character_output.size())
        # return

        if phase == 'train':
            optimizer.zero_grad()
            loss_gradient.backward()
            optimizer.step()

        # if i >= 10:
        #     break


    print('\nEpoch: {:d} \t Phase: {:s} \n'.format(epoch, phase))
    metric = print_metric('classification', classification_metric_dict, phase)
    if args.phase != 'train':
        print('metric = ', metric,'\n','\n')
        return
    if phase == 'val':
        if metric > p_dict['best_metric'][0]:
            p_dict['best_metric'] = [metric, epoch]
            save_model(p_dict)
            if 0:
            # if args.task == 'task2':
                preds = classification_metric_dict['preds'] 
                labels = classification_metric_dict['labels'] 
                times = classification_metric_dict['times'] 
                fl = open('/content/drive/MyDrive/sepsismodel/result/tauc_label.csv', 'w')
                fr = open('/content/drive/MyDrive/sepsismodel/result/tauc_result.csv', 'w')
                fl.write('adm_id,last_event_time,mortality\n')
                fr.write('adm_id,probability\n')
                for i, (p,l,t) in enumerate(zip(preds, labels, times)):
                    if i % 30:
                        continue
                    fl.write(str(i) + ',')
                    fl.write(str(t) + ',')
                    fl.write(str(int(l)) + '\n')

                    fr.write(str(i) + ',')
                    fr.write(str(p) + '\n')


        print('valid: metric: {:3.4f}\t epoch: {:d}\n'.format(metric, epoch))
        print('\t\t\t valid: best_metric: {:3.4f}\t epoch: {:d}\n'.format(p_dict['best_metric'][0], p_dict['best_metric'][1]))  
    else:
        print('train: metric: {:3.4f}\t epoch: {:d}\n'.format(metric, epoch))



def main():
    p_dict = dict() # All the parameters
    p_dict['args'] = args
    args.split_nn = args.split_num + args.split_nor * 3
    args.vocab_size = args.split_nn * 145 + 1
    print('vocab_size', args.vocab_size)

    ### load data
    print('read data ...')
    patient_time_record_dict = py_op.myreadjson('/content/drive/MyDrive/sepsismodel/runfiles/patient_time_record_dict.json')
    patient_master_dict = py_op.myreadjson('/content/drive/MyDrive/sepsismodel/runfiles/patient_master_dict.json')
    patient_label_dict = py_op.myreadjson('/content/drive/MyDrive/sepsismodel/runfiles/patient_label_dict.json')

    patient_train = list(json.load(open(os.path.join("/content/drive/MyDrive/sepsismodel/file/", args.task, 'train.json'))))
    patient_valid = list(json.load(open(os.path.join("/content/drive/MyDrive/sepsismodel/file/", args.task, 'val.json')))) 

    if len(patient_train) > len(patient_label_dict):
        # patients = patient_time_record_dict.keys()
        patients = list(patient_label_dict.keys())
        n = int(0.8 * len(patients))
        patient_train = patients[:n]
        patient_valid = patients[n:]

    print('data loading ...')
    train_dataset  = dataloader.DataSet(
                patient_train, 
                patient_time_record_dict,
                patient_label_dict,
                patient_master_dict, 
                args=args,
                phase='train')
    train_loader = DataLoader(
                dataset=train_dataset, 
                batch_size=args.batch_size,
                shuffle=True, 
                num_workers=2, 
                pin_memory=True)
    val_dataset  = dataloader.DataSet(
                patient_valid, 
                patient_time_record_dict,
                patient_label_dict,
                patient_master_dict, 
                args=args,
                phase='val')
    val_loader = DataLoader(
                dataset=val_dataset, 
                batch_size=args.batch_size,
                shuffle=False, 
                num_workers=2, 
                pin_memory=True)

    p_dict['train_loader'] = train_loader
    p_dict['val_loader'] = val_loader



    cudnn.benchmark = True
    net = LSTM(args)
    if args.gpu:
        net = net.cuda()
        p_dict['loss'] = loss.Loss().cuda()
    else:
        p_dict['loss'] = loss.Loss()

    parameters = []
    for p in net.parameters():
        parameters.append(p)
    optimizer = torch.optim.Adam(parameters, lr=args.lr)
    p_dict['optimizer'] = optimizer
    p_dict['model'] = net
    start_epoch = 0
    # args.epoch = start_epoch
    # print ('best_f1score' + str(best_f1score))

    p_dict['epoch'] = 0
    p_dict['best_metric'] = [0, 0]


    ### resume pretrained model
    if os.path.exists(args.resume):
        print('resume from model ' + args.resume)
        function.load_model(p_dict, args.resume)
        print('best_metric', p_dict['best_metric'])
        # return


    if args.phase == 'train':

        best_f1score = 0
        for epoch in range(p_dict['epoch'] + 1, args.epochs):
            p_dict['epoch'] = epoch
            for param_group in optimizer.param_groups:
                param_group['lr'] = args.lr
            train_eval(p_dict, 'train')
            train_eval(p_dict, 'val')


if __name__ == '__main__':
    main()


vocab_size 2031
read data ...
data loading ...


  0%|          | 0/3 [00:00<?, ?it/s]

input size torch.Size([32, 45, 72])


100%|██████████| 3/3 [00:01<00:00,  2.70it/s]



Epoch: 1 	 Phase: train 

classification
loss: 0.6974	 pos loss: 0.3414	 negloss: 0.3560
accuracy: 0.4162	 f1score: 0.4420	 recall: 0.5333	 precision: 0.3774


train: metric: 0.4420	 epoch: 1



...


Epoch: 19 	 Phase: train 

classification
loss: 0.6101	 pos loss: 0.4597	 negloss: 0.1504
accuracy: 0.7283	 f1score: 0.5437	 recall: 0.3733	 precision: 1.0000


train: metric: 0.5437	 epoch: 19



  0%|          | 0/1 [00:00<?, ?it/s]

input size torch.Size([18, 45, 72])


100%|██████████| 1/1 [00:00<00:00,  9.62it/s]


Epoch: 19 	 Phase: val 

classification
loss: 0.6007	 pos loss: 0.4372	 negloss: 0.1635
accuracy: 0.8023	 f1score: 0.5333	 recall: 0.3636	 precision: 1.0000


valid: metric: 0.6350	 epoch: 19

			 valid: best_metric: 0.7261	 epoch: 14

